In [ ]:
import tensorflow
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import keras
from tensorflow import image
import tensorflow_addons as tfa
import os
from collections import Counter


In [ ]:
batch_size = 32
img_height = 128
img_width = 128
AUTOTUNE = tf.data.experimental.AUTOTUNE


In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "train/",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "real_imgs/",
  #validation_split=1,
  #subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
#augment training data
def random_shift_img(x):
    if tf.random.uniform([])< 0.6:
        shift = tf.random.uniform([],minval=-250,maxval = 255)
        return x+shift
    else:
        return x
    

class RandomShift(layers.Layer):
    def __init__(self,dynamic = True, **kwargs):
        super().__init__(**kwargs)

    def call(self, x):
        return random_shift_img(x)


class Stretch(layers.Layer):
    def __init__(self,dynamic = True, **kwargs):
        super().__init__(**kwargs)

    def call(self, x):
        return tfa.image.transform(x, [0.5, 0, -250, 0.0, 1, 0.0, 0, 0.0])

    
    
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom((-0.5,0)),
    #to_grey
    RandomShift()
  ]
)
    

def prepare(ds):
    ds = ds.map(lambda x, y: (data_augmentation(x), y), 
                num_parallel_calls=AUTOTUNE)
    return ds.prefetch(buffer_size=AUTOTUNE)

train_ds = prepare(train_ds.unbatch().batch(1)).unbatch().batch(batch_size)


In [ ]:
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = 3


model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

In [ ]:
epochs = 10
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Testing Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Testing Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Testing Loss')
plt.legend(loc='upper right')
plt.title('Training and Testing Loss')
plt.show()

In [ ]:
#test NN on real images
real_img_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "real_imgs/",
  batch_size = 1,
  seed=123,
  image_size=(img_height, img_width))

#real_img_ds = grey_scale(real_img_ds)

In [ ]:
results = model.evaluate(real_img_ds)
